# December 04

Today's puzzle is very similar to the problem John and Rikk has recently worked on. 

We need to build a timeline of states, where a Elf is either awake or asleep. 

In [ ]:
import regex as re
import numpy as np
import pandas as pd

In [ ]:
# Sample data as provided in question
data= """
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up
""".strip().splitlines()

In [ ]:
# We're now going to start storing the data in a file to avoid having to paste into the main document
# Make sure you create a file with the name below and save the real problem output there. 
# If you want to run just the sample data, skip this block
with open("./04-kws.txt", "r") as FILE:
    data = FILE.read().strip().splitlines()

In [ ]:
data[:5]

In the real data the 'events' are not sorted, so before we do anything else, we need to sort the list. The wonderful thing about ISO8601 order date format is that the dates sort alphanumerically. Keep that in mind whenever putting a date in filenames!

So to put this in chronological order, we simply need to sort the list.

In [ ]:
data.sort()
data

In [ ]:
guard_on_duty = None

sleep_times = dict()

pattern_guard_change = re.compile(".*Guard #(\d+) begins shift")
pattern_date = re.compile("\[(\d\d\d\d-\d\d-\d\d) (\d\d):(\d\d)\].*")
for line in data:
    # Check if the current line is a guard change
    match = pattern_guard_change.match(line)
    if match:
        guard_on_duty = match[1]
        continue
        
    # If not a guard change, then it has to be an awake or sleep line
    match = pattern_date.match(line)
    date = match[1]
    minutes = int(match[3])
    
    date_key = "{}|{}".format(date,guard_on_duty)
    sleep_array = sleep_times.get(date_key, np.zeros(60))
        
    # If instead it's a guard that falls asleep
    if "asleep" in line:
        sleep_array[minutes:] = 1
    elif "wakes up" in line:
        sleep_array[minutes:] = 0
        
    sleep_times[date_key] = sleep_array

df_sleep = pd.DataFrame.from_dict(sleep_times,orient="index")

# Split date and elf into separate index columns
df_sleep["date"] = df_sleep.index.map(lambda x: x.split("|")[0])
df_sleep["elf"] = df_sleep.index.map(lambda x: x.split("|")[1])
df_sleep = df_sleep.set_index(["date","elf"])

df_sleep

First we want to find the elf that falls asleep the most

In [ ]:
total_sleep = df_sleep.groupby('elf').sum().sum(axis=1).sort_values(ascending=False)
total_sleep

In [ ]:
elf_with_max_sleep = total_sleep.head(1).keys()[0]
elf_with_max_sleep

In [ ]:
total_by_minute = df_sleep.groupby('elf').sum().reset_index()
total_by_minute

In [ ]:
df_elf = total_by_minute[total_by_minute["elf"] == elf_with_max_sleep]
del df_elf["elf"]
df_elf

In [ ]:
# Find the max minute 
df_elf_by_minute = df_elf.transpose()
df_elf_by_minute.columns = ['minutes_asleep']
df_elf_by_minute = df_elf_by_minute.sort_values(by='minutes_asleep', ascending=False)
df_elf_by_minute

In [ ]:

df_elf_by_minute["solution"] = df_elf_by_minute.index.astype(int) * int(elf_with_max_sleep)
df_elf_by_minute.head(1)

# Part 2

We now how to find the most common minute for each each

In [ ]:
# Find the most number of times an elf has been asleep on a particular minute
df_most_times_asleep = df_sleep.groupby('elf').sum().max(axis=1).sort_values(ascending=False)
df_most_times_asleep

In [ ]:
most_minutes = df_most_times_asleep.head(1).values[0]
elf = df_most_times_asleep.index[0]

"Elf {} was asleep {} times on a single minute".format(elf, most_minutes)

In [ ]:
df_sleep_by_minute = df_sleep.groupby("elf").sum().reset_index()
df_sleep_by_minute = df_sleep_by_minute[df_sleep_by_minute["elf"] == elf]
del df_sleep_by_minute["elf"]
df_sleep_by_minute

In [ ]:
df_sleep_by_minute2 = df_sleep_by_minute.transpose()
df_sleep_by_minute2.columns=["minutes_asleep"]
df_sleep_by_minute2["solution"] = df_sleep_by_minute2.index.astype(int) * int(elf)
df_sleep_by_minute2[df_sleep_by_minute2["minutes_asleep"] == most_minutes]